Configuration i 

----
Update python to 3.10.13 version

In [ ]:
%conda install --yes -c defaults -c conda-forge --update-all python=3.10.13

Configuration ii

---

Uninstall package fitz, which may cause import conflict

In [ ]:
%pip uninstall fitz

Configuration iii

---

Go to your conda's base directory(e.g. `D:\anaconda3`),under the path `D:\anaconda3\Lib\site-packages`, **delete** all folders which name contain "fitz" if there any

![jupyter](./path.png)

Configuration iv

---
Use pip to install the necessary packages instead of using conda

In [7]:
%pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.8/101.8 kB 2.5 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


Configuration v

---
press **Ctrl** + **SHFIT** + **P**, type **reload window** and select to reboot the kernel

Configuration vi

---
Following Section only need to be run at the first time

In [168]:
from aiohttp import ClientSession
import json, regex

headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:98.0) Gecko/20100101 Firefox/98.0"}
async def __constant_update():
    async with ClientSession(raise_for_status=True,headers=headers) as c:
        async with c.get("https://www.sec.gov/edgar/search/js/edgar_full_text_search.js") as res:
            _script = await res.text()

        with open("constants.py", "w", encoding="utf-8") as f:
            f.write("_FORMS = ")
            json.dump({
                form.pop("form"): form
                for form in eval(regex.search(
                    R"^const forms = (\[\r?\n(?: {4}\{.*?\},*\r?\n)*(?: {4}\{.*?\})\r?\n\])\.sort",
                    _script,
                    regex.MULTILINE
                )[1])
            }, f, indent=4)

            f.write('\n')

            f.write("_LOCATIONS = ")
            json.dump(dict(eval(regex.search(
                R"^const locationsArray = (\[\r?\n(?: {4}\[.*?\],\r?\n)*(?: {4}\[.*?\])\r?\n\]);",
                _script,
                regex.MULTILINE
            )[1])), f, indent=4)

            f.write('\n')
await __constant_update()

In [76]:
import asyncio
import json
import logging
from datetime import date
from itertools import zip_longest
from pathlib import Path
from typing import Any, Dict, Generator, List, Optional, Tuple, Union
from bs4 import BeautifulSoup
import pandas as pd
from aiohttp import ClientSession,ClientResponseError
from faker import Faker
import regex
from tenacity import retry,stop_after_attempt, wait_fixed,after_log,RetryError
#import retry
import time
from tqdm import *
from constants import _FORMS, _LOCATIONS
import os
import io
import fitz
_DISPLAY_NAME_REGEX = regex.compile(R"(.*) \(CIK (\d{10})\)", regex.V1)
_CC_REGEX = regex.compile(R"[\p{Cc}\p{Cf}]+", regex.V1)
_LOGGER = logging.getLogger(__name__)
_RETRY_SC = {403, 500, 502, 503, 504}
_DEFAULT_BYTE = b'{"took":530,"timed_out":false,"_shards":{"total":50,"successful":50,"skipped":0,"failed":0},"hits":{"total":{"value":0,"relation":"eq"},"max_score":null,"hits":[]},"aggregations":{"entity_filter":{"doc_count_error_upper_bound":0,"sum_other_doc_count":0,"buckets":[]},"sic_filter":{"doc_count_error_upper_bound":0,"sum_other_doc_count":0,"buckets":[]},"biz_states_filter":{"doc_count_error_upper_bound":0,"sum_other_doc_count":0,"buckets":[]},"form_filter":{"doc_count_error_upper_bound":0,"sum_other_doc_count":0,"buckets":[]}},"query":"{\\"query\\":{\\"bool\\":{\\"must\\":[{\\"match_phrase\\":{\\"doc_text\\":\\"data\\"}},{\\"match_phrase\\":{\\"doc_text\\":\\"breach\\"}}],\\"must_not\\":[],\\"should\\":[],\\"filter\\":[{\\"terms\\":{\\"ciks\\":[\\"0001199046\\"]}},{\\"range\\":{\\"file_date\\":{\\"gte\\":\\"2024-01-02\\",\\"lte\\":\\"2024-01-02\\"}}}]}},\\"from\\":0,\\"size\\":100,\\"aggregations\\":{\\"form_filter\\":{\\"terms\\":{\\"field\\":\\"root_form\\",\\"size\\":30}},\\"entity_filter\\":{\\"terms\\":{\\"field\\":\\"display_names.raw\\",\\"size\\":30}},\\"sic_filter\\":{\\"terms\\":{\\"field\\":\\"sics\\",\\"size\\":30}},\\"biz_states_filter\\":{\\"terms\\":{\\"field\\":\\"biz_states\\",\\"size\\":30}}}}"}'

# @retry(stop=stop_after_attempt(10), wait=wait_fixed(2),reraise=False)
# async def fetch(fetch_bar,semaphore,client,phrases,cik,end,forms,start='2001-01-01',range = 'custom',category= 'custom',entity=None,): #'https://efts.sec.gov/LATEST/search-index? 
             
#     q = " ".join(f"\"{phrase}\"" for phrase in phrases)
#     #forms = " ".join(form for  form in forms)
#     data = {'q':q,
#             'startdt':start,
#             'enddt':end,
#             'ciks':cik,
#             'dataRange':'custom',
#             'category':'custom',
#             'forms':forms}
#     url = 'https://efts.sec.gov/LATEST/search-index'
    
#     async with semaphore, client.request(method='get', url=url, params=data) as res:
#         await asyncio.sleep(1)
#         try:
#             if res.status == 200:
#                 result = await res.read()
#                 fetch_bar.update(1)
#                 return result
#         except  RetryError as e: #ClientResponseError or
#             _LOGGER.warning(f"{cik}-{start}-{end} query will be skipped: ({res.status}) {res.reason}")
#             fetch_bar.update(1)
#             return _DEFAULT_BYTE
    
@retry(wait=wait_fixed(1))
async def fetch(fetch_bar,semaphore,client,phrases,cik,end,forms,start='2001-01-01',range = 'custom',category= 'custom',entity=None,): #'https://efts.sec.gov/LATEST/search-index? 
             
    q = " ".join(f"\"{phrase}\"" for phrase in phrases)
    #forms = " ".join(form for  form in forms)
    data = {'q':q,
            'startdt':start,
            'enddt':end,
            'ciks':cik,
            'dataRange':'custom',
            'category':'custom',
            'forms':forms}
    url = 'https://efts.sec.gov/LATEST/search-index'
    async with semaphore,client.request(method='get',url=url,params = data) as res:
        await asyncio.sleep(1)
        if res.status == 200:
            result = await res.read()
            fetch_bar.update(1)
            return result#await res.json()
        raise ValueError(f"Status Code = {res.status}")
    
def _concat_to_url(cik: str, adsh: str, filename: str) -> str:
    return f"https://www.sec.gov/Archives/edgar/data/{cik}/{adsh}/{filename}"

@retry(stop=stop_after_attempt(6), wait=wait_fixed(2),after=after_log(_LOGGER, logging.WARNING),reraise=False)#, 
async def _download(semaphore: asyncio.Semaphore, row_index, df,client,keywords,download_bar):
    url = df.loc[row_index,"url"]
    ext = df.loc[row_index,"file_ext"]
    if ext not in ["htm","pdf","txt"]:
        _LOGGER.warning(f"Unkown extension in row [{row_index}]")
        tem = f"paragrah{1}"
        df.loc[row_index,tem] = 'Unknow extension'
        return download_bar.update(1)
        
    async with semaphore, client.get(url) as res:
        await asyncio.sleep(1)
        if res.status == 404:
            _LOGGER.warning(f"Url: {url} is not exist: ({res.status}) {res.reason}. Skipping download.")
            return download_bar.update(1)
        if res.status == 403 or res.status == 429:
            _LOGGER.warning(f"Reach request limitation: {url}: ({res.status}) {res.reason}.\n Restart to crawl after 10 minutes. ")
            await asyncio.sleep(630)
            start_time = time.time()
            while True:
                confirm = input("Please open 'https://www.sec.gov/edgar/search/' in your browser, press 'ok' to crawl continue if it available")
                if confirm.lower() == "ok":
                    if res.ok:
                        _LOGGER.warning("Manully confirm. Continue to crawl ")
                        break
                    else:
                        _LOGGER.warning(f"Fail to Manully confirm: ({res.status}) {res.reason} ")
                        start_time = time.time()
                        
                current_time = time.time()
                elapsed_time = current_time - start_time
                if elapsed_time >= 180 :
                    if res.ok:
                        _LOGGER.warning("Automatically confirm. Continue to crawl ")
                        break
                    else:
                        start_time = time.time()
                        _LOGGER.warning("Fail to automate confirm. Will retry again after 10 mintues")
                        await asyncio.sleep(600)
                        start_time = time.time()
                        
        if res.ok:
            try:
                html = await res.read()
                if ext == "htm":
                    paragraphs = extract_html(html,keywords)
                elif ext == "pdf":
                    paragraphs = extract_pdf(html,keywords)
                elif ext == "txt":
                    paragraphs = extract_txt(html,keywords)
                if not paragraphs:
                    _LOGGER.warning(f"No content extracted from {url}")
                    tem = f"paragrah{1}"
                    df.loc[row_index, tem] = 'No content extracted'
                    return download_bar.update(1)
    
                for num in range(len(paragraphs)):
                    tem = f"paragrah{num + 1}"
                    df.loc[row_index,tem] = str(paragraphs[num])
                return download_bar.update(1)
            except RetryError:
                _LOGGER.warning(f"Retring download url: {url} at the row: {row_index}")
                return download_bar.update(1)

    _LOGGER.warning(f"Failed to download {url}: ({res.status}) {res.reason}")
    tem = f"paragrah{1}"
    df.loc[row_index, tem] = 'Download skipped'
    #raise Exception(f"Failed to download {url}: ({res.status}) {res.reason}")
    return download_bar.update(1)



def _parse_display_name(s: str, cik: str):
    if s is not None and (m := _DISPLAY_NAME_REGEX.fullmatch(s)):
        if (scik := m[2]) != cik:
            _LOGGER.warning(f"mismatched CIK: {scik} (parsed from \"{s}\") v.s. {cik}")
        return m[1], scik
    return s, cik

def _parse_hit(hit: Dict[str, Any]): 
    _id = hit["_id"]
    source = hit["_source"]
    adsh, filename = _id.split(':')
    filename_main, filename_ext = filename.rsplit('.', 1)
    xsl = source["xsl"]
    
    if xsl and filename_ext.lower() == "xml":
        filename_main = f"{xsl}/{filename_main}"
    filename = f"{filename_main}.{filename_ext}"

    file_nums = source["file_num"]
    film_nums = source["film_num"]
    rows = pd.DataFrame((
        [_id, *_parse_display_name(display_name, cik), str(loc).split(",")[0], _LOCATIONS.get(code), file_num, film_num]
        for display_name, cik, loc, code, file_num, film_num in zip_longest(
            source["display_names"],
            source["ciks"],
            source["biz_locations"],
            source["biz_states"], #source["inc_states"] if source["inc_states"] else 
            file_nums if isinstance(file_nums, list) else [file_nums] if file_nums else (),
            film_nums if isinstance(film_nums, list) else [film_nums] if film_nums else ()
        ) 
    ), columns=["id", "entity_name", "cik", "located", "incorporated", "file_num", "film_num"], copy=False)#, dtype=str
    form = source["form"]
    root_form = source["root_form"]
    form_title = ""
    if root_form in _FORMS:
        form_title = f" ({_FORMS[root_form]['title']})"
    file_type = source["file_type"]
    if not file_type:
        file_type = source["file_description"]
    if not file_type:
        file_type = filename
    ciks = rows.loc[0,"cik"]
    info = pd.DataFrame({
        "entity_name":rows['entity_name'],
        "id": _id,
        "form_file": f"{form}{form_title}{'' if form == file_type else f' {file_type}'}",
        "file_date": source["file_date"],
        "period_ending": source.get("period_ending", None),
        "file_ext": filename_ext,
        "url": _concat_to_url(ciks, adsh.replace('-', ''), filename),
        "parser": None#getattr(parsers, f"_parse_{filename_ext.lower()}", None)
    },copy=False,dtype=str)#, dtype=object
    
    result = pd.merge(rows,info,how="left",on="id")
    return result.reset_index(drop=True)

#len_check = lambda chunk,sub_delimiter: [chunk] if len(chunk) <=3000 else chunk.split(sub_delimiter)

def extract_html(html, keywords):
    matching_paragraphs = [regex.sub(r"(\s+)"," ",para).strip() 
                           for para in BeautifulSoup(html, "lxml",from_encoding='utf-8').get_text('\n\n').split('\n\n') 
                           if any(keyword in para.lower().replace("-"," ").replace("\n"," ") 
                            for keyword in keywords)]
    return matching_paragraphs

def extract_pdf(fetch,keywords):
    pdf_stream = io.BytesIO(fetch)
    pdf_document = fitz.open(stream=pdf_stream,filetype="pdf")

    total = [para.replace("\n"," ") for page in range(pdf_document.page_count) 
            for lists in pdf_document[page].get_text("blocks") 
            for  para in lists 
            if any(keyword in str(para).lower().replace("-"," ") for keyword in keywords) ]
    return total

def extract_txt(bytes,keywords):
    txt = bytes.decode('utf-8').split('\n\n')
    txt = [i.replace("\n"," ")for i in txt]
    matching_paragraphs = [paragraph for paragraph in txt if any(keyword in paragraph.lower().replace("-"," ") for keyword in keywords)]
    return matching_paragraphs

    
def chop_time_interval(start_date,end_date,frequency = "2D"):
    interval = pd.interval_range(pd.Timestamp(start_date) ,pd.Timestamp(end_date),closed="both",freq=frequency)
    date_pattern = '%Y-%m-%d'
    intervals = [(interval[i].left.strftime((date_pattern)), interval[i].right.strftime((date_pattern))) for i in range(len(interval)) if i % 2 ==0 ]
    return intervals

def decode(byte):
    total_hits = json.loads(byte.decode('utf-8'))['hits']['total']['value']
    if total_hits >= 100:
        _LOGGER.warning(f"Numbers opf Hits/entries >= 100. Try to reduce the interval of data range to avoid data missing.")
    hits =  json.loads(byte.decode('utf-8'))["hits"]["hits"]
    #print(hits)
    return hits

def CIK(ciks):
    if isinstance(ciks, Path):
        try:
            with open(ciks, "r", encoding="UTF-8") as f:
                try:
                    _ciks = [f"{int(cik):010}" for cik in f.read().splitlines()]
                except ValueError as e:
                    raise ValueError(f"{ciks} contains invalid CIKs") from e
        except IOError as e:
            raise ValueError(f"{ciks} is not a valid file") from e
    # if it's an iterable of values, treat all values as CIKs
    elif isinstance(ciks, list):
        try:
            _ciks = [f"{int(cik):010}" for cik in ciks]
        except ValueError as e:
            raise ValueError(f"{ciks} is not a valid CIK list") from e
    # if it's a single string, consider it as a single CIK
    elif isinstance(ciks, str):
        try:
            _ciks = [f"{int(ciks):010}"]
        except ValueError as e:
            raise ValueError(f"{ciks} is not a valid CIK") from e
    # same as previous with the preferred (int) type
    elif isinstance(ciks, int):
        _ciks = [f"{ciks:010}"]
    if _ciks == []:
        _ciks = ""
    return _ciks
    
async def main(_PHRASES,
               _FILING_TYPES,
               _DATE_START,
               _DATE_END,
               _CIKS_PER_QUERY, 
               _CIKS,
               _buffer_chunk_size,
               df,headers,
               _OUTPUT_NAME,
               _OUTPUT_FORMAT,
               _DATA_RANGE_INTERVAL="2D"):
    if _OUTPUT_FORMAT not in ["excel","xlsx","csv"]:
        raise TypeError(f"{_OUTPUT_FORMAT} is not a valid extension. Options extension [excel,xlsx,csv].")
    semaphore = asyncio.Semaphore(10)
    if _FILING_TYPES == []:
        _FILING_TYPES = [""]
    #-------------- Crawl --------

    async with ClientSession(raise_for_status=False, headers=headers) as client :
        #------Fetch--------
        if _CIKS:
            total = len(_CIKS) * len(_FILING_TYPES)
            with tqdm(        
                total=total) as fetch_bar:
                print("Starting fetch...")
                fetch_tasks = [
                        asyncio.create_task(fetch(
                            semaphore=semaphore,
                            client=client,
                            phrases=_PHRASES,
                            cik=cik,
                            start=_DATE_START,
                            end=_DATE_END,
                            forms=form,
                            fetch_bar=fetch_bar
                        ))
                        for  form in _FILING_TYPES for cik in _CIKS
                    ]
                fetched_data = await asyncio.gather(*fetch_tasks)
        else:
            intervals = chop_time_interval(_DATE_START,_DATE_END,_DATA_RANGE_INTERVAL)
            with tqdm(total=len(intervals)) as fetch_bar:
                fetch_tasks = [asyncio.create_task(fetch(
                                    semaphore=semaphore,
                                    client=client,
                                    phrases=_PHRASES,
                                    cik="",
                                    start=start,
                                    end=end,
                                    forms=_FILING_TYPES,
                                    fetch_bar=fetch_bar
                                )) for start,end in intervals]
                
                fetched_data = await asyncio.gather(*fetch_tasks)
        df = pd.concat([df] + 
                    [_parse_hit(hit) for data in fetched_data 
                     for hit in decode(data)],
                    ignore_index=True)
        df.drop_duplicates(subset="id",inplace=True)
        df = df.sample(frac=1.0)
        df.reset_index(drop=True,inplace=True)
        #-----Docs download-------
        print(f"fetch completed and collected [{df.shape[0]}] of docs,starting download docs..")
        total = df.shape[0]
        if total == 0:
            print("Fetch completed with 0 result. Now existing.")
            return
        with tqdm(total=total) as download_bar:
            for index in range(0,total,_buffer_chunk_size):
                index_range = list(range(index,min(index+_buffer_chunk_size,df.shape[0])))
                download_tasks = [
                    asyncio.create_task(_download(
                        semaphore=semaphore,
                        client=client,
                        df = df,
                        row_index=row,
                        download_bar=download_bar,
                        keywords=_PHRASES)
                    )for row in index_range
                    ]
                downloaded = await asyncio.gather(*download_tasks)
                


    del df["parser"] 
    del df['id']
    if _OUTPUT_FORMAT in ["excel","xlsx"]:
        df.to_excel(f"{_OUTPUT_NAME}.xlsx")
    elif _OUTPUT_FORMAT.lower() == "csv":      
        df.to_csv(f"{_OUTPUT_NAME}.csv")
    print(f"Data have been export at {os.getcwd()}") 
    return df

## Parameters

This section defines all customisable parameters.
- **df** (`pandas dataframe`) : Output data frame.

- **headers** (`str`) : A string of user's email address for declaration.

- **PHRASES** (`List[str]`): A list of keywords or phrases to search for.

- **DATE_START** & **DATE_END** (both `date`): As indicated by the name. But it should conform to the ISO time format, i.e., YYYY-MM-DD as shown in the example.

- **DATA_RANGE_INTERVAL** (`str`): A string to set up a data range interval for per query. Enable when do not providing an specific CIKs input, i.e., "1D","3D" and "**X**D". Reducing the interval will result in more queires have to be made. Recommanded set as not more than "2D" since may return more than 100 enties per query and casue data missing.

- **FILING_TYPES** (`List[str]`): A list of filling types.

- **CIKS** (`Optional[Union[Path, int, str, List[Union[int, str]]]]`): A list of CIKs in no more than 10 digits, or it can be a path to the file containing all CIKs for the query.

- **BUFFER_CHUNK_SIZE**: The maximum number of files allowed to be cached in the memory.

- **OUTPUT_NAME**: The file name without the suffix of the output file.

- **OUTPUT_FORMAT** (`enumerate["excel","xlsx","csv"]`): The file format of the output file.

In [77]:
import time
import warnings
warnings.filterwarnings("ignore")
T0 = time.time()
df = pd.DataFrame()
headers = {"User-Agent":"a1835067@student.adelaide.edu.au"}
_PHRASES = ["data breach", "cyber security"]
_DATA_RANGE_INTERVAL = "2D" # 
#_FILING_TYPES = [""]#,
_FILING_TYPES = [""]
_DATE_START = "2001-01-01"
_DATE_END = "2023-12-27"
_CIKS_PER_QUERY = 10
_BUFFER_CHUNK_SIZE = 10
_CIKS =  [] #Input from a list or a path,,"0001653481"
#_CIKS = CIK(Path("sample_input_file.txt"))[2400:] ####Please mannually select N here for Testing for the first N CIKs###
_OUTPUT_NAME = "20240102_noCKIs_full"
# _OUTPUT_FORMAT = "csv"
_OUTPUT_FORMAT = "excel"


df = await main(
    _PHRASES,
    _FILING_TYPES,
    _DATE_START,
    _DATE_END,
    _CIKS_PER_QUERY,
    _CIKS,
    _BUFFER_CHUNK_SIZE,
    df,
    headers,
    _OUTPUT_NAME,
    _OUTPUT_FORMAT,
    _DATA_RANGE_INTERVAL
)
 
END = time.time()
print("--"*10,
      f"All tasks completed! Time Cost:{round((END-T0)/60,1)} minutes ",sep='\n')

  0%|          | 0/2099 [00:00<?, ?it/s]

100%|██████████| 2099/2099 [14:52<00:00,  2.35it/s]
mismatched CIK: 0001199046 (parsed from "Pear Tree Funds  (CIK 0001199046)") v.s. 0000722885


fetch completed and collected [7571] of docs,starting download docs..


100%|██████████| 7571/7571 [1:56:34<00:00,  1.08it/s]


Data have been export at /mnt/d/summer_research
--------------------
All tasks completed! Time Cost:132.0 minutes 


In [64]:
def decode(byte):
    total_hits = json.loads(byte.decode('utf-8'))['hits']['total']['value']
    # if total_hits == 0:
    #     return None
    hits =  json.loads(byte.decode('utf-8'))["hits"]["hits"]
    #print(hits)
    return hits
decode(b'')

JSONDecodeError: Expecting value: line 1 column 1 (char 0)